# Deep Learning Tutorial

#### Abram Hindle
#### <abram.hindle@ualberta.ca>
#### http://softwareprocess.ca/

Slides stolen gracefully from Ben Zittlau

Slide content under CC-BY-SA 4.0 and MIT License for source code or the same license as Python3 or Keras. Slide Source code is MIT License as well.




## Start

First off lets get a useful Python environment!

Please install theanets and bpython.

`````
pip install --user keras
pip install --user tensorflow
pip install --user scikit-learn
`````

Get a recent version of this presentation:

```
git clone https://github.com/abramhindle/keras-tutorial.git
```


# Intro
### What is machine learning?

Building a function from data to classify, predict, group, or represent data.



# Intro
### Machine Learning

There are a few kinds of tasks or functions that could help us here.

* Classification: given some input, predict the class that it belongs
  to. Given a point is it in the red or in the blue?
* Regression: Given a point what will its value be? In the case of a
  function with a continuous or numerous discrete outputs it might be
  appropriate.
* Representation: Learn a smaller representation of the input
  data. E.g. we have 300 features lets describe them in a 128-bit hash.




# Intro
### Motivational Example

Imagine we have this data:

![2 crescent slices](images/slice.png "A function we want to learn
 f(x,y) -> z where z is red")

[See src/genslice.py to see how we made it.](src/genslice.py)



In [2]:
# purpose: make a slight difference of circles be the dataset to learn.
import numpy as np
gY, gX = np.meshgrid(np.arange(1000)/1000.0,np.arange(1000)/1000.0)

def intersect_circle(cx,cy,radius):
    equation = (gX - float(cx)) ** 2 + (gY - float(cy)) ** 2
    matches = equation < (radius**3) 
    return matches

# rad = 0.1643167672515498
rad = 0.3
x = intersect_circle(0.5,0.5,rad) ^ intersect_circle(0.51,0.51,rad)

def plotit(x):
    import matplotlib.pyplot as plt
    plt.imshow(x)
    plt.savefig('new-slice.png') # was slice.png
    plt.imshow(x)
    plt.savefig('new-slice.pdf') # was slice.pdf
    plt.show()

# plotit(x)

def mkcol(x):
    return x.reshape((x.shape[0]*x.shape[1],1))

# make the data set
big = np.concatenate((mkcol(gX),mkcol(gY),mkcol(1*x)),axis=1)
np.savetxt("new-big-slice.csv", big, delimiter=",")

# make a 50/50 data set
nots = big[big[0:,2]==0.0,]
np.random.shuffle(nots)
nots = nots[0:1000,]
trues = big[big[0:,2]==1.0,]
np.random.shuffle(trues)
trues = trues[0:1000,]
small = np.concatenate((trues,nots))
np.savetxt("new-small-slice.csv", small, delimiter=",")


# Intro
### Make your own function

``` python
def in_circle(x,y,cx,cy,radius):
    return (x - float(cx)) ** 2 + (y - float(cy)) ** 2 < radius**2

def mysolution(pt,outer=0.3):
    return in_circle(pt[0],pt[1],0.5,0.5,outer) and not in_circle(pt[0],pt[1],0.5,0.5,0.1)
```

```
>>> myclasses = np.apply_along_axis(mysolution,1,test[0])
>>> print "My classifier!"
My classifier!
>>> print "%s / %s " % (sum(myclasses == test[1]),len(test[1]))
181 / 200 
>>> print theautil.classifications(myclasses,test[1])
[('tp', 91), ('tn', 90), ('fp', 19), ('fn', 0)]
```



# Intro 
### An example classifier

1-NN: 1 Nearest Neighbor.

Given the data, we produce a function that
outputs the CLASS of the nearest neighbour to the input data.

Whoever is closer, is the class. 3-NN is 3-nearest neighbors whereby
we use voting of the 3 neighbors instead.



# Intro
### An example classifier: 1-NN

[src/slice-classifier.py](src/slice-classifier.py)

``` python
def euclid(pt1,pt2):
    return sum([ (pt1[i] - pt2[i])**2 for i in range(0,len(pt1)) ])

def oneNN(data,labels):
    def func(input):
        distance = None
        label = None
        for i in range(0,len(data)):
            d = euclid(input,data[i])
            if distance == None or d < distance:
                distance = d
                label = labels[i]
        return label
    return func
```




# Intro
### An example classifier: 1-NN

``` python
>>> learner = oneNN(train[0],train[1])
>>> 
>>> oneclasses = np.apply_along_axis(learner,1,test[0])
>>> print "1-NN classifier!"
1-NN classifier!
>>> print "%s / %s " % (sum(oneclasses == test[1]),len(test[1]))
198 / 200 
>>> print theautil.classifications(oneclasses,test[1])
[('tp', 91), ('tn', 107), ('fp', 2), ('fn', 0)]

```

1-NN has great performance in this example, but it uses Euclidean
distance and the dataset is really quite biased to the positive
classes.

Thus we showed a simple learner that classifies data.



# Intro

* That's really interesting performance and it worked but will it
  scale and continue to work?

* 1-NN doesn't work for all problems. And it is dependent on linear
  relationships.

* What if our problem is non-linear?




In [3]:
#
# The MIT License (MIT)
# 
# Copyright (c) 2016 Abram Hindle <hindle1@ualberta.ca>, Leif Johnson <leif@lmjohns3.com>
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# first off we load up some modules we want to use
import keras
import scipy
import math
import numpy as np
import numpy.random as rnd
import logging
import sys
import collections
import theautil

# setup logging
logging.basicConfig(stream = sys.stderr, level=logging.INFO)

mupdates = 1000
data = np.loadtxt("small-slice.csv", delimiter=",")
inputs  = data[0:,0:2].astype(np.float32)
outputs = data[0:,2:3].astype(np.int32)

theautil.joint_shuffle(inputs,outputs)

train_and_valid, test = theautil.split_validation(90, inputs, outputs)
train, valid = theautil.split_validation(90, train_and_valid[0], train_and_valid[1])
print("Train: %s Valid: %s"%(len(train),len(valid)))
print("Train X: %s Y^:%s"%(train[0].shape,train[1].shape))
print("Valid X: %s  Y^:%s"%(valid[0].shape,valid[1].shape))


Train: 2 Valid: 2
Train X: (1620, 2) Y^:(1620, 1)
Valid X: (180, 2)  Y^:(180, 1)


In [4]:

def linit(x):
    return x.reshape((len(x),))

mltrain = (train[0],linit(train[1]))
mlvalid = (valid[0],linit(valid[1]))
mltest  = (test[0] ,linit(test[1]))

# my solution
def in_circle(x,y,cx,cy,radius):
    return (x - float(cx)) ** 2 + (y - float(cy)) ** 2 < radius**2

def mysolution(pt,outer=0.3):
    return in_circle(pt[0],pt[1],0.5,0.5,outer) and not in_circle(pt[0],pt[1],0.5,0.5,0.1)

# apply my classifier
myclasses = np.apply_along_axis(mysolution,1,mltest[0])
print("My classifier!")
print("%s / %s " % (sum(myclasses == mltest[1]),len(mltest[1])))
print(theautil.classifications(myclasses,mltest[1]))

My classifier!
175 / 200 
[('tp', 102), ('tn', 73), ('fp', 25), ('fn', 0)]


In [5]:
def euclid(pt1,pt2):
    return sum([ (pt1[i] - pt2[i])**2 for i in range(0,len(pt1)) ])

def oneNN(data,labels):
    def func(input):
        distance = None
        label = None
        for i in range(0,len(data)):
            d = euclid(input,data[i])
            if distance == None or d < distance:
                distance = d
                label = labels[i]
        return label
    return func

learner = oneNN(mltrain[0],mltrain[1])

oneclasses = np.apply_along_axis(learner,1,mltest[0])
print("1-NN classifier!")
print("%s / %s " % (sum(oneclasses == mltest[1]),len(mltest[1])))
print(theautil.classifications(oneclasses,mltest[1]))


1-NN classifier!
200 / 200 
[('tp', 102), ('tn', 98), ('fp', 0), ('fn', 0)]



# Intro

* Neural networks are popular
   * Creating AI for Go
   * Labeling Images with cats and dogs
   * Speech Recognition
   * Text summarization
   * [Guitar Transcription](https://peerj.com/preprints/1193.pdf)
   * Learn audio from video[1](https://archive.org/details/DeepLearningBitmaptoPCM/)[2](http://softwareprocess.es/blog/blog/2015/08/10/deep-learning-bitmaps-to-pcm/)

* Neural networks can not only classify, but they can create content,
  they can have complicated outputs.

* Neural networks are generative!


# Intro
### Machine Learning: Neural Networks

Neural networks or "Artificial Neural Networks" are a flexible class
of non-linear machine learners. They have been found to be quite
effective as of late.

Neural networks are composed of neurons. These neurons try to emulate
biological neurons in the most metaphorical of senses. Given a set of
inputs they produce an output.




## Neurons

Neurons have functions.

* Rectified Linear Units have been shown to train quite well and
  achieve good results. By they aren't easier to differentiate.
  f(x) = max(0,x)
* Sigmoid functions are slow and were the classical neural network
  neuron, but have fallen out of favour. They will work when nothing
  else will. f(x) = 1/(1 + e^-x)
* Softplus is a RELU that is slower to compute but differentiable.
  f(x) = ln(1 + e^x)




## Neurons

![Rectifier and Sigmoid and Softplus](images/Rectifier_and_softplus_functions.svg)






## Neurons

The inputs to a neural network? The outputs of connected nodes times
their weight + a bias.

neuron(inputs) = neuron_f( sum(weights * inputs) + bias  )

![Neuron example](images/neuron.png)




## Multi-layer perceptron

Single hidden layer neural network.

![Multi-layer perceptron](images/20160208141015.png)






## Deep Learning

There's nothing particularly crazy about deep learning other than it has more hidden layers.

These hidden layers allow it to compute state and address the intricacies of complex functions. But each hidden layer adds a lot of search space.




## Deep Learning

![Deep network, multiple layers](images/20160208141143.png)






## Search

How do we find the different weights?

Well we need to search a large space. A 2x3x2 network will have 2*3*2
weights + 5 biases (3 hidden, 2 output) resulting in 17
parameters. That's already a large search space.

Most search algorithms measure their error at a certain point
(difference between prediction and actual) and then choose a direction
in their search space to travel. They do this by sampling points
around themselves in order to compute a gradient or slope and then
follow the slope around.

Here's a 3D demo of different search algorithms.

[Different Search Parameters](http://www.robertsdionne.com/bouncingball/)





## Let's deep learn on our problem

![2 crescent slices](images/slice.png "A function we want to learn
 f(x,y) -> z where z is red")

Please open [slice-classifier](./src/slice-classifier.py) and a python
interpreter such as bpython. Search for Part 3 around line 100.




In [20]:

print('''
########################################################################
# Part 3. Let's start using neural networks!
########################################################################
''')

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train[1])
train_y = enc.transform(train[1])
valid_y = enc.transform(valid[1])
test_y = enc.transform(test[1])

print(train[0].shape)
print(train[1].shape)
print(train_y.shape)



########################################################################
# Part 3. Let's start using neural networks!
########################################################################

(1620, 2)
(1620, 1)
(1620, 2)


In [21]:
# rerunning this will produce different results
# try different combos here
net = Sequential()
net.add(Dense(16,input_shape=(2,),activation="sigmoid"))
net.add(Dense(16,activation="relu"))
net.add(Dense(2,activation="softmax"))
opt = SGD(lr=0.1)
# opt = Adam(lr=0.1)
net.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
history = net.fit(train[0], train_y, validation_data=(valid[0], valid_y),
	            epochs=100, batch_size=16)


Train on 1620 samples, validate on 180 samples
Epoch 1/100
1620/1620 [==============================] - 0s 89us/step - loss: 0.7018 - acc: 0.4994 - val_loss: 0.7809 - val_acc: 0.4889
Epoch 2/100
1620/1620 [==============================] - 0s 93us/step - loss: 0.6986 - acc: 0.4932 - val_loss: 0.6981 - val_acc: 0.4889
Epoch 3/100
1620/1620 [==============================] - 0s 92us/step - loss: 0.6941 - acc: 0.4988 - val_loss: 0.6939 - val_acc: 0.5111
Epoch 4/100
1620/1620 [==============================] - 0s 97us/step - loss: 0.6944 - acc: 0.4988 - val_loss: 0.6927 - val_acc: 0.4889
Epoch 5/100
1620/1620 [==============================] - 0s 90us/step - loss: 0.6936 - acc: 0.5111 - val_loss: 0.6967 - val_acc: 0.4389
Epoch 6/100
1620/1620 [==============================] - 0s 87us/step - loss: 0.6911 - acc: 0.5185 - val_loss: 0.6991 - val_acc: 0.4889
Epoch 7/100
1620/1620 [==============================] - 0s 91us/step - loss: 0.6908 - acc: 0.5278 - val_loss: 0.6913 - val_acc: 0.4167
E

1620/1620 [==============================] - 0s 90us/step - loss: 0.3043 - acc: 0.8938 - val_loss: 0.3079 - val_acc: 0.8556
Epoch 61/100
1620/1620 [==============================] - 0s 94us/step - loss: 0.2983 - acc: 0.8920 - val_loss: 0.2517 - val_acc: 0.9111
Epoch 62/100
1620/1620 [==============================] - 0s 89us/step - loss: 0.2940 - acc: 0.8932 - val_loss: 0.4370 - val_acc: 0.7444
Epoch 63/100
1620/1620 [==============================] - 0s 92us/step - loss: 0.2911 - acc: 0.8914 - val_loss: 0.2736 - val_acc: 0.8889
Epoch 64/100
1620/1620 [==============================] - 0s 93us/step - loss: 0.2961 - acc: 0.8901 - val_loss: 0.4881 - val_acc: 0.6944
Epoch 65/100
1620/1620 [==============================] - 0s 87us/step - loss: 0.2959 - acc: 0.8858 - val_loss: 0.2989 - val_acc: 0.8833
Epoch 66/100
1620/1620 [==============================] - 0s 96us/step - loss: 0.2887 - acc: 0.8988 - val_loss: 0.2380 - val_acc: 0.9111
Epoch 67/100
1620/1620 [==============================

In [22]:
print("Learner on the test set")
score = net.evaluate(test[0], test_y)
print("Scores: %s" % score)
predictit = net.predict(test[0])
print(predictit.shape)
print(predictit[0:10,])
classify = net.predict_classes(test[0])

print("%s / %s " % (np.sum(classify == mltest[1]),len(mltest[1])))
print(collections.Counter(classify))
print(theautil.classifications(classify,mltest[1]))


Learner on the test set
200/200 [==============================] - 0s 54us/step
Scores: [0.2766148710250855, 0.895]
(200, 2)
[[9.4758965e-02 9.0524101e-01]
 [1.2368786e-01 8.7631214e-01]
 [9.9979770e-01 2.0228619e-04]
 [1.0000000e+00 1.9776802e-08]
 [1.6064729e-01 8.3935273e-01]
 [9.9999821e-01 1.8081136e-06]
 [9.9463791e-01 5.3620674e-03]
 [9.9937552e-01 6.2449591e-04]
 [1.1010743e-01 8.8989258e-01]
 [9.9959230e-01 4.0771632e-04]]
179 / 200 
Counter({1: 123, 0: 77})
[('tp', 102), ('tn', 77), ('fp', 21), ('fn', 0)]


Let's try this on unseen data.

In [23]:

def real_function(pt):
    rad = 0.1643167672515498
    in1 = in_circle(pt[0],pt[1],0.5,0.5,rad)
    in2 = in_circle(pt[0],pt[1],0.51,0.51,rad)
    return in1 ^ in2

print("And now on more unseen data that isn't 50/50")

bigtest = np.random.uniform(size=(3000,2)).astype(np.float32)
biglab = np.apply_along_axis(real_function,1,bigtest).astype(np.int32)

classify = net.predict_classes(bigtest)
print("%s / %s " % (sum(classify == biglab),len(biglab)))
print(collections.Counter(classify))
print(theautil.classifications(classify,biglab))


And now on more unseen data that isn't 50/50
2498 / 3000 
Counter({0: 2471, 1: 529})
[('tp', 27), ('tn', 2471), ('fp', 502), ('fn', 0)]




## Now let's discuss posing problems for neural networks

* Scaling inputs: Scaling can sometimes help, so can
  standardization. This means constraining values or re-centering
  them. It depends on your problem and it is worth trying.

* E.g. min max scaling:

``` python
def min_max_scale(data):
    '''scales data by minimum and maximum values between 0 and 1'''
    dmin = np.min(data)
    return (data - dmin)/(np.max(data) - dmin)
```



## The problem

* [posing.py](src/posing.py) tries to show the problem of taking
  random input data and determine what distribution it comes from.
  That is what function can produce these random values.

* Let's open up [posing.py](src/posing.py) and get an interpreter
  going.



In [24]:
# Demonstration of how to pose the problem and how different formulations
# lead to different results!
#
# The MIT License (MIT)
# 
# Copyright (c) 2016 Abram Hindle <hindle1@ualberta.ca>, Leif Johnson <leif@lmjohns3.com>
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# first off we load up some modules we want to use
import keras
import scipy
import math
import numpy as np
import numpy.random as rnd
import logging
import sys
from numpy.random import power, normal, lognormal, uniform
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import theanets

# What are we going to do?
# - we're going to generate data derived from 4 different distributions
# - we're going to scale that data
# - we're going to create a RBM (1 hidden layer neural network)
# - we're going to train it to classify data as belonging to one of these distributions

# maximum number of iterations before we bail
mupdates = 1000

# setup logging
logging.basicConfig(stream = sys.stderr, level=logging.INFO)

# how we pose our problem to the deep belief network matters.

# lets make the task easier by scaling all values between 0 and 1
def min_max_scale(data):
    '''scales data by minimum and maximum values between 0 and 1'''
    dmin = np.min(data)
    return (data - dmin)/(np.max(data) - dmin)

# how many samples per each distribution
bsize    = 100 

# poor man's enum
LOGNORMAL=0
POWER=1
NORM=2
UNIFORM=3



## Experiment 1

* Given 1 single sample what distribution does it come from?




In [27]:
print('''
########################################################################
# Experiment 1: can we classify single samples?
#
#
#########################################################################
''')

def make_dataset1():
    '''Make a dataset of single samples with labels from which distribution they come from'''
    # now lets make some samples 
    lns      = min_max_scale(lognormal(size=bsize)) #log normal
    powers   = min_max_scale(power(0.1,size=bsize)) #power law
    norms    = min_max_scale(normal(size=bsize))    #normal
    uniforms = min_max_scale(uniform(size=bsize))    #uniform
    # add our data together
    data = np.concatenate((lns,powers,norms,uniforms))
    
    # concatenate our labels
    labels = np.concatenate((
        (np.repeat(LOGNORMAL,bsize)),
        (np.repeat(POWER,bsize)),
        (np.repeat(NORM,bsize)),
        (np.repeat(UNIFORM,bsize))))
    tsize = len(labels)
    
    # make sure dimensionality and types are right
    data = data.reshape((len(data),1))
    data = data.astype(np.float32)
    labels = labels.astype(np.int32)
    labels = labels.reshape((len(data),))
    
    return data, labels, tsize

# this will be the training data and validation data
data, labels, tsize = make_dataset1()


# this is the test data, this is kept separate to prove we can
# actually work on the data we claim we can.
#
# Without test data, you might just have great performance on the
# train set.
test_data, test_labels, _ = make_dataset1()



########################################################################
# Experiment 1: can we classify single samples?
#
#
#########################################################################



In [46]:
# utilities

# now lets shuffle
# If we're going to select a validation set we probably want to shuffle
def joint_shuffle(arr1,arr2):
    assert len(arr1) == len(arr2)
    indices = np.arange(len(arr1))
    np.random.shuffle(indices)
    arr1[0:len(arr1)] = arr1[indices]
    arr2[0:len(arr2)] = arr2[indices]

# our data and labels are shuffled together
joint_shuffle(data,labels)

def split_validation(percent, data, labels):
    ''' 
    split_validation splits a dataset of data and labels into
    2 partitions at the percent mark
    percent should be an int between 1 and 99
    '''
    s = int(percent * len(data) / 100)
    tdata = data[0:s]
    vdata = data[s:]
    tlabels = labels[0:s]
    vlabels = labels[s:]
    return ((tdata,tlabels),(vdata,vlabels))

# make a validation set from the train set
train1, valid1 = split_validation(90, data, labels)

print(train1[0].shape)
print(train1[1].shape)

enc1 = OneHotEncoder(handle_unknown='ignore')
enc1.fit(train1[1].reshape(len(train1[1]),1))
train1_y = enc1.transform(train1[1].reshape(len(train1[1]),1))
print(train1_y.shape)
valid1_y = enc1.transform(valid1[1].reshape(len(valid1[1]),1))
print(valid1_y.shape)
test1_y = enc1.transform(test_labels.reshape(len(test_labels),1))
print(test1_y.shape)

(360, 1)
(360,)
(360, 4)
(40, 4)
(400, 4)


In [47]:
# build our classifier

print("We're building a MLP of 1 input layer node, 4 hidden layer nodes, and an output layer of 4 nodes. The output layer has 4 nodes because we have 4 classes that the neural network will output.")
cnet = Sequential()
cnet.add(Dense(4,input_shape=(1,),activation="sigmoid"))
cnet.add(Dense(4,activation="softmax"))
copt = SGD(lr=0.1)
# opt = Adam(lr=0.1)
cnet.compile(loss="categorical_crossentropy", optimizer=copt, metrics=["accuracy"])
history = cnet.fit(train1[0], train1_y, validation_data=(valid1[0], valid1_y),
	            epochs=100, batch_size=16)

#score = cnet.evaluate(test_data, test_labels)
#print("Scores: %s" % score)
classify = cnet.predict_classes(test_data)
print(theautil.classifications(classify,test_labels))
score = cnet.evaluate(test_data, test1_y)
print("Scores: %s" % score)

#print "%s / %s " % (sum(cnet.classify(data) == labels),tsize)
#print "%s / %s " % (sum(cnet.classify(test_data) == test_labels),tsize)

We're building a MLP of 1 input layer node, 4 hidden layer nodes, and an output layer of 4 nodes. The output layer has 4 nodes because we have 4 classes that the neural network will output.
Train on 360 samples, validate on 40 samples
Epoch 1/100
360/360 [==============================] - 0s 92us/step - loss: 1.4406 - acc: 0.2556 - val_loss: 1.4422 - val_acc: 0.2000
Epoch 2/100
360/360 [==============================] - 0s 89us/step - loss: 1.3945 - acc: 0.2444 - val_loss: 1.4080 - val_acc: 0.2000
Epoch 3/100
360/360 [==============================] - 0s 94us/step - loss: 1.3883 - acc: 0.2694 - val_loss: 1.3882 - val_acc: 0.2000
Epoch 4/100
360/360 [==============================] - 0s 95us/step - loss: 1.3879 - acc: 0.1944 - val_loss: 1.3941 - val_acc: 0.2500
Epoch 5/100
360/360 [==============================] - 0s 98us/step - loss: 1.3857 - acc: 0.2694 - val_loss: 1.3910 - val_acc: 0.2000
Epoch 6/100
360/360 [==============================] - 0s 90us/step - loss: 1.3828 - acc: 0.255

Epoch 60/100
360/360 [==============================] - 0s 96us/step - loss: 1.2106 - acc: 0.3944 - val_loss: 1.0952 - val_acc: 0.5000
Epoch 61/100
360/360 [==============================] - 0s 96us/step - loss: 1.2068 - acc: 0.4111 - val_loss: 1.0919 - val_acc: 0.4750
Epoch 62/100
360/360 [==============================] - 0s 97us/step - loss: 1.2039 - acc: 0.4139 - val_loss: 1.0878 - val_acc: 0.6000
Epoch 63/100
360/360 [==============================] - 0s 95us/step - loss: 1.2017 - acc: 0.4056 - val_loss: 1.0898 - val_acc: 0.4000
Epoch 64/100
360/360 [==============================] - 0s 89us/step - loss: 1.1990 - acc: 0.4000 - val_loss: 1.0725 - val_acc: 0.5750
Epoch 65/100
360/360 [==============================] - 0s 88us/step - loss: 1.1969 - acc: 0.4556 - val_loss: 1.0692 - val_acc: 0.4000
Epoch 66/100
360/360 [==============================] - 0s 91us/step - loss: 1.1956 - acc: 0.4333 - val_loss: 1.0660 - val_acc: 0.5000
Epoch 67/100
360/360 [==============================] -


## Experiment 2

* Given 40 samples what distribution does it come from?





## Experiment 3

* Given 40 sorted samples what distribution does it come from?


## Experiment 4

* Given 40 histogrammed samples what distribution does it come from?



## Representation: Inputs

* For discrete values consider discrete inputs neurons. E.g. if you have 3 letters are your input you should have 3 * 26 input numerous. Each neuron is "one-hot" -- 1 neuron is set to 1 to indicate that 1 discerete value. An input of AAA would be: 
  * 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
* ZZZ would be 
  * 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1





## Representation: Inputs

* For groups of elements consider representing them as their counts.
* E.g. 3 cats, 4 dogs, 1 car as: 3 4 1 on 3 input neurons.
* Neural networks work well with distributions as inputs and distributions as outputs



## Representation: Words

* Words can be represented as word counts where by your vector is the count of each word per document -- you might have a large vocabulary so watch out!
* n-grams are popular too with one-hot encoding



## Representaiton: Images

* Each neuron can represent a pixel represented from 0 to 1
* You can have images as output too!





## Representation: Outputs

* Do not ask the neural network to distingush discrete values on 1 neuron. Don't expect 1 neuron to output 0.25 for A and 0.9 for B and 1.0 for C. Use 3 neurons!
* Distribution outputs are good
* Interpretting the output is fine for regression problems




## References

* [Theanets Documentation](https://theanets.readthedocs.org/en/stable/)
* [A Practical Guide to TrainingRestricted Boltzmann Machines](https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf)
* [MLP](http://deeplearning.net/tutorial/mlp.html#mlp)
* [Deep Learning Tutorials](http://www.iro.umontreal.ca/~pift6266/H10/notes/deepintro.html)
* [Deep Learning Tutorials](http://deeplearning.net/tutorial/)
* [Coursera: Hinton's Neural Networks for Machine Learning](https://www.coursera.org/course/neuralnets)
* [The Next Generation of Neural Networks](https://www.youtube.com/watch?v=AyzOUbkUf3M)
* [Geoffrey Hinton: "Introduction to Deep Learning & Deep Belief Nets"](https://www.youtube.com/watch?v=GJdWESd543Y)
* Bengio's Deep Learning
  [(1)](https://www.youtube.com/watch?v=JuimBuvEWBg)[(2)](https://www.youtube.com/watch?v=Fl-W7_z3w3o)
* [Nvidia's Deep Learning tutorials](https://developer.nvidia.com/deep-learning-courses
)
* [Udacity Deep Learning MOOC](https://www.udacity.com/course/deep-learning--ud730)
